---

***Filename***: lti_coefficients.ipynb

---

***Description***: This notebook computes the coefficients for the IIR digital filter of the *Boss DS-1* **Passive Filter** stage

---
***Author***: Alexandros Iliadis


***Project***: MSc Thesis

***Date***: June 2025

---

### Execution Initialization

In [33]:
# Import Modules (Built-In)
import os
import sys
import time

# Import Modules (User-Defined)
sys.path.append(os.path.abspath('Modules'))
import symbolic as sym
import utilities as utils

# Import Modules (Third-Party)
import sympy as sp
from IPython.display import display
from IPython.display import Markdown as MD

# Start Timer
start_time = time.perf_counter()
utils.printHeader("iir_coefficients.ipynb",line_break = False)

                    Executing File: iir_coefficients.ipynb | 25/06/2025 18:46:11


### Circuit Topology Description

In [34]:
# Input Sources
Vin,Iin = sp.symbols('V_in,I_in')

# Resistors
R15,R16,R17,R_tone,R_level = sp.symbols('R_15,R_16,R_17,R_tone,R_level')
R15_eq = sp.Eq(R15,2.2e3)
R16_eq = sp.Eq(R16,6.8e3)
R17_eq = sp.Eq(R17,6.8e3)
R_tone_eq = sp.Eq(R_tone,20e3)
R_level_eq = sp.Eq(R_level,100e3)
resistors = [R15_eq,R16_eq,R17_eq,R_tone_eq,R_level_eq]

# Variable Resistors
x = sp.symbols('x')
Ra,Rb = sp.symbols('R_a,R_b') 
Ra_eq = sp.Eq(Ra,R_tone*(1 - x))
Rb_eq = sp.Eq(Rb,R_tone*x)
variable_resistors = [Ra_eq,Rb_eq]

# Capacitors
C11,C12 = sp.symbols('C_11,C_12')
C11_eq = sp.Eq(C11,22e-9)
C12_eq = sp.Eq(C12,0.1e-6)
capacitors = [C11_eq,C12_eq]

# Impedances
s = sp.symbols('s')
Za,Zb = sp.symbols('Z_a,Z_b') 
Za_eq = sp.Eq(Za,sp.together(R15 + 1/(s*C11)))
Zb_eq = sp.Eq(Zb,1/(s*C12))
impedances = [Za_eq,Zb_eq]

# Node Voltages
Vout,Va,Vb,Vs = sp.symbols('V_out,V_a,V_b,V_s')
V = [Vout,Vs,Va,Vb,Iin]

# System Equations
eq1 = sp.Eq((Vb - Vout)/Rb,(Vout - Va)/Ra + Vout/R_level)
eq2 = sp.Eq((Vs - Vb)/R16,Vb/Zb + (Vb - Vout)/Rb)
eq3 = sp.Eq((Vs - Va)/Za + (Vout - Va)/Ra,Va/R17)
eq4 = sp.Eq(Iin,(Vs - Vb)/R16 + (Vs - Va)/Za)
eq5 = sp.Eq(Vs,Vin)
system_equations = [eq1,eq2,eq3,eq4,eq5]

# Cell Output
display(MD("---"))
display(MD("Resistors:"))
for element in resistors:
    display(element)
display(MD("---"))
display(MD("Variable Resistors:"))
for element in variable_resistors:
    display(element)
display(MD("---"))
display(MD("Capacitors:"))
for element in capacitors:
    display(element)
display(MD("---"))
display(MD("Impedances:"))
for element in impedances:
    display(element)
display(MD("---"))
display(MD("System Equations:"))
for eq in system_equations:
    display(eq)
display(MD("---"))

---

Resistors:

Eq(R_15, 2200.0)

Eq(R_16, 6800.0)

Eq(R_17, 6800.0)

...

Eq((-V_b + V_s)/R_16, V_b/Z_b + (V_b - V_out)/R_b)

Eq((-V_a + V_s)/Z_a + (-V_a + V_out)/R_a, V_a/R_17)

Eq(I_in, (-V_a + V_s)/Z_a + (-V_b + V_s)/R_16)

Eq(V_s, V_in)

---

### Modified Nodal Analysis

In [35]:
# System Matrices (GV = I)
G,I = sym.systemToMatrixForm(system_equations,V)

# System Solution
system_solution = sym.solveLinearSystem(G,I,V)
V_eq = sym.formEquations(V,system_solution)

# Cell Output
display(MD("---"))
display(MD("Matrix G:"))
display(G)
display(MD("Vector V:"))
display(sp.Matrix(V))
display(MD("Vector I:"))
display(I)
display(MD("---"))
display(MD("System Solution:"))
for eq in V_eq:
    display(eq)
display(MD("---"))

---

Matrix G:

Matrix([
[-1/R_level - 1/R_b - 1/R_a,               0,                   1/R_a,                   1/R_b, 0],
[                     1/R_b,          1/R_16,                       0, -1/Z_b - 1/R_b - 1/R_16, 0],
[                     1/R_a,           1/Z_a, -1/Z_a - 1/R_a - 1/R_17,                       0, 0],
[                         0, -1/Z_a - 1/R_16,                   1/Z_a,                  1/R_16, 1],
[                         0,               1,                       0,                       0, 0]])

Vector V:

Matrix([
[V_out],
[  V_s],
[  V_a],
[  V_b],
[ I_in]])

...

Eq(V_s, V_in)

Eq(V_a, R_17*V_in*(R_16*R_a*R_b + R_16*R_a*R_level + R_16*R_a*Z_b + R_16*R_b*R_level + R_16*R_level*Z_b + R_a*R_b*Z_b + R_a*R_level*Z_b + R_b*R_level*Z_b + R_level*Z_a*Z_b)/(R_16*R_17*R_a*R_b + R_16*R_17*R_a*R_level + R_16*R_17*R_a*Z_b + R_16*R_17*R_b*R_level + R_16*R_17*R_b*Z_a + R_16*R_17*R_level*Z_a + R_16*R_17*R_level*Z_b + R_16*R_17*Z_a*Z_b + R_16*R_a*R_b*Z_a + R_16*R_a*R_level*Z_a + R_16*R_a*Z_a*Z_b + R_16*R_b*R_level*Z_a + R_16*R_level*Z_a*Z_b + R_17*R_a*R_b*Z_b + R_17*R_a*R_level*Z_b + R_17*R_b*R_level*Z_b + R_17*R_b*Z_a*Z_b + R_17*R_level*Z_a*Z_b + R_a*R_b*Z_a*Z_b + R_a*R_level*Z_a*Z_b + R_b*R_level*Z_a*Z_b))

Eq(V_b, V_in*Z_b*(R_16*R_17*R_level + R_17*R_a*R_b + R_17*R_a*R_level + R_17*R_b*R_level + R_17*R_b*Z_a + R_17*R_level*Z_a + R_a*R_b*Z_a + R_a*R_level*Z_a + R_b*R_level*Z_a)/(R_16*R_17*R_a*R_b + R_16*R_17*R_a*R_level + R_16*R_17*R_a*Z_b + R_16*R_17*R_b*R_level + R_16*R_17*R_b*Z_a + R_16*R_17*R_level*Z_a + R_16*R_17*R_level*Z_b + R_16*R_17*Z_a*Z_b + R_16*R_a*R_b*Z_a + R_16*R_a*R_level*Z_a + R_16*R_a*Z_a*Z_b + R_16*R_b*R_level*Z_a + R_16*R_level*Z_a*Z_b + R_17*R_a*R_b*Z_b + R_17*R_a*R_level*Z_b + R_17*R_b*R_level*Z_b + R_17*R_b*Z_a*Z_b + R_17*R_level*Z_a*Z_b + R_a*R_b*Z_a*Z_b + R_a*R_level*Z_a*Z_b + R_b*R_level*Z_a*Z_b))

Eq(I_in, V_in*(R_16*R_17*R_b + R_16*R_17*R_level + R_16*R_17*Z_b + R_16*R_a*R_b + R_16*R_a*R_level + R_16*R_a*Z_b + R_16*R_b*R_level + R_16*R_level*Z_b + R_17*R_a*R_b + R_17*R_a*R_level + R_17*R_a*Z_b + R_17*R_b*R_level + R_17*R_b*Z_a + R_17*R_b*Z_b + R_17*R_level*Z_a + R_17*Z_a*Z_b + R_a*R_b*Z_a + R_a*R_b*Z_b + R_a*R_level*Z_a + R_a*R_level*Z_b + R_a*Z_a*Z_b + R_b*R_level*Z_a + R_b*R_level*Z_b + R_level*Z_a*Z_b)/(R_16*R_17*R_a*R_b + R_16*R_17*R_a*R_level + R_16*R_17*R_a*Z_b + R_16*R_17*R_b*R_level + R_16*R_17*R_b*Z_a + R_16*R_17*R_level*Z_a + R_16*R_17*R_level*Z_b + R_16*R_17*Z_a*Z_b + R_16*R_a*R_b*Z_a + R_16*R_a*R_level*Z_a + R_16*R_a*Z_a*Z_b + R_16*R_b*R_level*Z_a + R_16*R_level*Z_a*Z_b + R_17*R_a*R_b*Z_b + R_17*R_a*R_level*Z_b + R_17*R_b*R_level*Z_b + R_17*R_b*Z_a*Z_b + R_17*R_level*Z_a*Z_b + R_a*R_b*Z_a*Z_b + R_a*R_level*Z_a*Z_b + R_b*R_level*Z_a*Z_b))

---

### Continuous-Time Transfer Function

In [36]:
# s-Domain Representation
tf_s = Vout/Vin
tf_s = sym.substituteSymbols(tf_s,sym.extractEquations(V_eq,Vout))
tf_s = sym.substituteSymbols(tf_s,impedances + variable_resistors)
tf_s,beta_s_eq,alpha_s_eq = sym.rationalFraction(tf_s,symbol = s,num_prefix = '\\beta',den_prefix = '\\alpha')

# Transfer Function
Hs = sp.symbols('H(s)')
Hs_eq = sp.Eq(Hs,tf_s)

# Coefficients
s_coeffs = [sp.Eq(eq.lhs,sym.groupBySymbol(eq.rhs,x)) for eq in beta_s_eq + alpha_s_eq]

# Cell Output
display(MD("---"))
display(MD("Transfer Function:"))
display(Hs_eq)
display(MD("---"))
display(MD("Coefficients:"))
for eq in s_coeffs:
    display(eq)
display(MD("---"))

---

Transfer Function:

Eq(H(s), (\beta_0 + \beta_1*s + \beta_2*s**2)/(\alpha_0 + \alpha_1*s + \alpha_2*s**2))

---

Coefficients:

...

Eq(\beta_2, -C_11*C_12*R_16*R_17*R_level*R_tone*x)

Eq(\alpha_0, -R_16*R_17 - R_16*R_level - R_16*R_tone - R_17*R_level - R_level*R_tone + R_tone**2*x**2 + x*(R_16*R_tone - R_17*R_tone - R_tone**2))

Eq(\alpha_1, -C_11*R_15*R_16*R_17 - C_11*R_15*R_16*R_level - C_11*R_15*R_16*R_tone - C_11*R_15*R_17*R_level - C_11*R_15*R_level*R_tone - C_11*R_16*R_17*R_level - C_11*R_16*R_17*R_tone - C_11*R_17*R_level*R_tone - C_12*R_16*R_17*R_level - C_12*R_16*R_level*R_tone + x**2*(C_11*R_15*R_tone**2 + C_11*R_17*R_tone**2 + C_12*R_16*R_tone**2) + x*(C_11*R_15*R_16*R_tone - C_11*R_15*R_17*R_tone - C_11*R_15*R_tone**2 + C_11*R_16*R_17*R_tone - C_11*R_17*R_tone**2 - C_12*R_16*R_17*R_tone - C_12*R_16*R_tone**2))

Eq(\alpha_2, -C_11*C_12*R_15*R_16*R_17*R_level - C_11*C_12*R_15*R_16*R_level*R_tone - C_11*C_12*R_16*R_17*R_level*R_tone + x**2*(C_11*C_12*R_15*R_16*R_tone**2 + C_11*C_12*R_16*R_17*R_tone**2) + x*(-C_11*C_12*R_15*R_16*R_17*R_tone - C_11*C_12*R_15*R_16*R_tone**2 - C_11*C_12*R_16*R_17*R_tone**2))

---

### Discrete-Time Transfer Function

In [37]:
# z-Domain Representation
c,z_inv = sp.symbols('c,z^{-1}')
s_eq = sp.Eq(s,c*(1 - z_inv)/(1 + z_inv))
tf_z = sym.substituteSymbols(tf_s,s_eq)
tf_z,beta_z_eq,alpha_z_eq = sym.rationalFraction(tf_z,symbol = z_inv,num_prefix = 'B',den_prefix = 'A')

# Transfer Function
Hz = sp.symbols('H(z)')
Hz_eq = sp.Eq(Hz,tf_z)

# Coefficients
z_coeffs = [sp.Eq(eq.lhs,sym.groupBySymbol(sym.substituteSymbols(eq,s_coeffs).rhs,x)) for eq in beta_z_eq + alpha_z_eq]

# Cell Output
display(MD("---"))
display(MD("Transfer Function:"))
display(Hz_eq)
display(MD("---"))
display(MD("Coefficients:"))
for eq in z_coeffs:
    display(eq)
display(MD("---"))

---

Transfer Function:

Eq(H(z), (B_0 + B_1*z^{-1} + B_2*z^{-1}**2)/(A_0 + A_1*z^{-1} + A_2*z^{-1}**2))

---

Coefficients:

...

Eq(B_2, C_11*R_15*R_17*R_level*c + C_11*R_15*R_level*R_tone*c + C_11*R_16*R_17*R_level*c + C_11*R_17*R_level*R_tone*c - R_17*R_level - R_level*R_tone + x*(-C_11*C_12*R_16*R_17*R_level*R_tone*c**2 - C_11*R_15*R_level*R_tone*c + R_level*R_tone))

Eq(A_0, -C_11*C_12*R_15*R_16*R_17*R_level*c**2 - C_11*C_12*R_15*R_16*R_level*R_tone*c**2 - C_11*C_12*R_16*R_17*R_level*R_tone*c**2 - C_11*R_15*R_16*R_17*c - C_11*R_15*R_16*R_level*c - C_11*R_15*R_16*R_tone*c - C_11*R_15*R_17*R_level*c - C_11*R_15*R_level*R_tone*c - C_11*R_16*R_17*R_level*c - C_11*R_16*R_17*R_tone*c - C_11*R_17*R_level*R_tone*c - C_12*R_16*R_17*R_level*c - C_12*R_16*R_level*R_tone*c - R_16*R_17 - R_16*R_level - R_16*R_tone - R_17*R_level - R_level*R_tone + x**2*(C_11*C_12*R_15*R_16*R_tone**2*c**2 + C_11*C_12*R_16*R_17*R_tone**2*c**2 + C_11*R_15*R_tone**2*c + C_11*R_17*R_tone**2*c + C_12*R_16*R_tone**2*c + R_tone**2) + x*(-C_11*C_12*R_15*R_16*R_17*R_tone*c**2 - C_11*C_12*R_15*R_16*R_tone**2*c**2 - C_11*C_12*R_16*R_17*R_tone**2*c**2 + C_11*R_15*R_16*R_tone*c - C_11*R_15*R_17*R_tone*c - C_11*R_15*R_tone**2*c + C_11*R_16*R_17*R_tone*c - C_11*R_17*R_tone**2*c - C_12*R_16*R_17*R_tone*c - C_12*R_16*R_tone**2*c + R_16*R_tone - R_17*R_tone - R_tone**2))

Eq(A_1, 2*C_11*C_12*R_15*R_16*R_17*R_level*c**2 + 2*C_11*C_12*R_15*R_16*R_level*R_tone*c**2 + 2*C_11*C_12*R_16*R_17*R_level*R_tone*c**2 - 2*R_16*R_17 - 2*R_16*R_level - 2*R_16*R_tone - 2*R_17*R_level - 2*R_level*R_tone + x**2*(-2*C_11*C_12*R_15*R_16*R_tone**2*c**2 - 2*C_11*C_12*R_16*R_17*R_tone**2*c**2 + 2*R_tone**2) + x*(2*C_11*C_12*R_15*R_16*R_17*R_tone*c**2 + 2*C_11*C_12*R_15*R_16*R_tone**2*c**2 + 2*C_11*C_12*R_16*R_17*R_tone**2*c**2 + 2*R_16*R_tone - 2*R_17*R_tone - 2*R_tone**2))

Eq(A_2, -C_11*C_12*R_15*R_16*R_17*R_level*c**2 - C_11*C_12*R_15*R_16*R_level*R_tone*c**2 - C_11*C_12*R_16*R_17*R_level*R_tone*c**2 + C_11*R_15*R_16*R_17*c + C_11*R_15*R_16*R_level*c + C_11*R_15*R_16*R_tone*c + C_11*R_15*R_17*R_level*c + C_11*R_15*R_level*R_tone*c + C_11*R_16*R_17*R_level*c + C_11*R_16*R_17*R_tone*c + C_11*R_17*R_level*R_tone*c + C_12*R_16*R_17*R_level*c + C_12*R_16*R_level*R_tone*c - R_16*R_17 - R_16*R_level - R_16*R_tone - R_17*R_level - R_level*R_tone + x**2*(C_11*C_12*R_15*R_16*R_tone**2*c**2 + C_11*C_12*R_16*R_17*R_tone**2*c**2 - C_11*R_15*R_tone**2*c - C_11*R_17*R_tone**2*c - C_12*R_16*R_tone**2*c + R_tone**2) + x*(-C_11*C_12*R_15*R_16*R_17*R_tone*c**2 - C_11*C_12*R_15*R_16*R_tone**2*c**2 - C_11*C_12*R_16*R_17*R_tone**2*c**2 - C_11*R_15*R_16*R_tone*c + C_11*R_15*R_17*R_tone*c + C_11*R_15*R_tone**2*c - C_11*R_16*R_17*R_tone*c + C_11*R_17*R_tone**2*c + C_12*R_16*R_17*R_tone*c + C_12*R_16*R_tone**2*c + R_16*R_tone - R_17*R_tone - R_tone**2))

---

In [38]:
# Evaluate Expressions
z_coeffs_eval = [sym.substituteSymbols(eq,resistors + capacitors) for eq in z_coeffs]

# Cell Output
display(MD("---"))
display(MD("Coefficients (Evaluated):"))
for eq in z_coeffs_eval:
    display(eq)
display(MD("---"))

---

Coefficients (Evaluated):

Eq(B_0, -530640.0*c + x*(-203.456*c**2 + 96800.0*c + 2000000000.0) - 2680000000.0)

Eq(B_1, x*(406.912*c**2 + 4000000000.0) - 5360000000.0)

Eq(B_2, 530640.0*c + x*(-203.456*c**2 - 96800.0*c + 2000000000.0) - 2680000000.0)

Eq(A_0, -291.66016*c**2 - 2415118.016*c + x**2*(53.856*c**2 + 351200.0*c + 400000000.0) + x*(-58.332032*c**2 - 423334.4*c - 400000000.0) - 3542240000.0)

Eq(A_1, 583.32032*c**2 + x**2*(800000000.0 - 107.712*c**2) + x*(116.664064*c**2 - 800000000.0) - 7084480000.0)

Eq(A_2, -291.66016*c**2 + 2415118.016*c + x**2*(53.856*c**2 - 351200.0*c + 400000000.0) + x*(-58.332032*c**2 + 423334.4*c - 400000000.0) - 3542240000.0)

---

### Execution Conclusion

In [39]:
# End Timer
runtime = utils.calculateElapsedTime(start_time,unit = 's')
utils.printFooter(runtime,line_break = False)

                          Execution Runtime: 3.733 s | 25/06/2025 18:46:15
